# Test avec MLP (jutse pour voir)

In [1]:
import os

In [2]:
os.getcwd()

'/home/franzele/Desktop/univ_lille/sd/facial_emotion_classification_sd/scripts'

In [3]:
os.chdir("..")

In [4]:
PATH_TO_DATA = "CK+_lands/CK+_centered"
PATH_TO_LABEL = "CK+_lands/CK+/emotion.csv"

In [5]:
import pandas as pd

In [6]:
def load_last_faces(data, labels, only_labelled=True):
    def fuse_df(base_df, visage_dir, tmp_id, data, target=-1):
        coords = {str(i):x for i, x in enumerate(data.to_numpy())}
        new_line = {
            "subject": visage_dir,
            "file": tmp_id,
        }
        new_line = new_line | coords
        new_line = pd.DataFrame(new_line, index=[0])

        # new_line["coords"] = new_line.iloc[:, 2:].to_numpy().tolist()

        # new_line = new_line.drop([str(i) for i in range(136)], axis=1)
        return pd.concat([base_df, new_line])

    last_faces = pd.DataFrame()
    labels = pd.read_csv(labels, delimiter=";")

    for visage_dir in os.listdir(data):
        visage_data = pd.read_csv(os.path.join(data, visage_dir, "omlands.csv"), delimiter=";", header=None)
        tmp_id = visage_data.iloc[0, 0]
        for row_id in visage_data.index:
            if visage_data.iloc[row_id, 0] != tmp_id:
                last_faces = fuse_df(last_faces, visage_dir, tmp_id, visage_data.iloc[row_id-1, 1:])
                tmp_id = visage_data.iloc[row_id, 0]
        last_faces = fuse_df(last_faces, visage_dir, tmp_id, visage_data.iloc[row_id-1, 1:])


    last_faces = pd.merge(last_faces, labels, on=["subject", "file"], how="inner" if only_labelled else "outer")
    
    return last_faces

In [7]:
data = load_last_faces(PATH_TO_DATA, PATH_TO_LABEL)

## Training

In [8]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [9]:
X_train, X_test, y_train, y_test = train_test_split(data.loc[:, "0":"135"], data["emotion"])

In [10]:
cross_val_score(MLPClassifier(), data.loc[:, "0":"135"], data["emotion"]).mean()

0.39863067292644755

<h1 style="color:red"><b>ATTENTION : </b> le prochain bloc de code utilisera tous les processeurs disponibles</h1>
(pour désactiver ça, mettre `n_jobs` à None)

In [11]:
param_grid = {
    "hidden_layer_sizes": [100, 500, 1000],
    "activation": ["logistic", "tanh", "relu"],
    "alpha": [0.0001, 0.001, 0.01, 0.1],
    "learning_rate_init": [0.1, 0.01, 0.001, 0.0001],
    "max_iter": [200, 500, 1000]
}

grid_search = GridSearchCV(MLPClassifier(), param_grid=param_grid, cv=5, n_jobs=-1)
grid_search.fit(X_train, y_train)

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/usr/lib/python3/dist-packages/scipy

GridSearchCV(cv=5, estimator=MLPClassifier(), n_jobs=-1,
             param_grid={'activation': ['logistic', 'tanh', 'relu'],
                         'alpha': [0.0001, 0.001, 0.01, 0.1],
                         'hidden_layer_sizes': [100, 500, 1000],
                         'learning_rate_init': [0.1, 0.01, 0.001, 0.0001],
                         'max_iter': [200, 500, 1000]})

In [12]:
grid_search.best_estimator_

MLPClassifier(activation='logistic', alpha=0.001, hidden_layer_sizes=1000,
              learning_rate_init=0.0001, max_iter=1000)

In [15]:
best_mlp = MLPClassifier(hidden_layer_sizes=1000, learning_rate_init=0.0001, alpha=0.001, activation="logistic", max_iter=1000)

In [16]:
cross_val_score(best_mlp, data.loc[:, "0":"135"], data["emotion"]).mean()

0.7618544600938967